In [25]:
#필요한 라이브러리 호출
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
#모델 불러옴
model_path = 'model/mobilenet_7.h5'
model = load_model(model_path)

In [27]:
def map_prediction_to_emotion(predictions):
    emotion_map = {
        0: 'anger',     
        1: 'disgust',  
        2: 'fear',      
        3: 'happy',     
        4: 'sadness',   
        5: 'surprise',  
        6: 'neutral'    #affectnet 7class 분류
    }
    mapped_predictions = []
    for pred in predictions:
        mapped_emotion = emotion_map[pred]
        if mapped_emotion in ['disgust', 'fear', 'surprise']:
            mapped_emotion = 'panic'
        if mapped_emotion == 'neutral':
            mapped_emotion = None
        mapped_predictions.append(mapped_emotion)
    return np.array(mapped_predictions)

In [28]:
#이미지 증강 
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    'img/img/train',
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 5995 images belonging to 4 classes.


In [32]:
#예측 수행
predictions = model.predict(data_generator)
predicted_labels = np.argmax(predictions, axis=1)
mapped_predictions = map_prediction_to_emotion(predicted_labels)

# 실제 라벨 가져오기
true_labels = data_generator.classes
true_labels_str = np.array([os.path.basename(os.path.dirname(filepath)) for filepath in data_generator.filepaths])

188/188 [==============================] - 186s 987ms/step


In [33]:
# 예측된 라벨과 실제 라벨 비교
incorrect_indices = np.where(mapped_predictions != true_labels_str)[0]

# 잘못 라벨링된 파일을 저장할 딕셔너리
mislabeled_files = {
    'happy': [],
    'anger': [],
    'sadness': [],
    'panic': []
}

In [34]:
#딕셔너리 채우기
for i in incorrect_indices:
    filename = data_generator.filenames[i]
    predicted_emotion = mapped_predictions[i]
    if predicted_emotion:
        mislabeled_files[predicted_emotion].append(filename)

In [35]:
output_dir = 'mislabeled_files'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 파일명 저장
for emotion in mislabeled_files:
    with open(os.path.join(output_dir, f'mislabeled_{emotion}.txt'), 'w') as file:
        for filename in mislabeled_files[emotion]:
            file.write(f"{filename}\n")